In [1]:
# Experiment 2 Script for COVID-19 Forecasting
# Change path to the root directory of the project
import os
os.chdir("../../")

# Description: This script contains the code for the second experiment in the project, 
# forecasting COVID-19 MVBeds using various RNN models and hyperparameter tuning with Simulated Annealing.

# Imports for handling data
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
from itertools import cycle

# Imports for machine learning
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse

# Imports for visualization
import plotly.express as px
import plotly.graph_objects as go

# Progress bar
from tqdm.autonotebook import tqdm
tqdm.pandas()

# Local imports for data loaders and models
from src.utils import plotting_utils
from src.dl.dataloaders import TimeSeriesDataModule
from src.dl.multivariate_models import SingleStepRNNConfig, SingleStepRNNModel, Seq2SeqConfig, Seq2SeqModel, RNNConfig
from src.transforms.target_transformations import AutoStationaryTransformer

# Set seeds for reproducibility
pl.seed_everything(42)
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

torch.set_float32_matmul_precision('high')

# Set default plotly template
import plotly.io as pio
pio.templates.default = "plotly_white"

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import logging

# Set logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

Global seed set to 42


In [2]:
# Utility Functions

def format_plot(fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text=ylabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text=xlabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
    )
    return fig

def mase(actual, predicted, insample_actual):
    mae_insample = np.mean(np.abs(np.diff(insample_actual)))
    mae_outsample = np.mean(np.abs(actual - predicted))
    return mae_outsample / mae_insample

def forecast_bias(actual, predicted):
    return np.mean(predicted - actual)

def plot_forecast(pred_df, forecast_columns, selected_area, forecast_display_names=None, save_path=None):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)

    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = px.colors.qualitative.D3  # Use a colorblind-friendly palette
    act_color = colors[0]
    colors = cycle(colors[1:])

    fig = go.Figure()

    # Actual data plot
    fig.add_trace(
        go.Scatter(
            x=pred_df[mask].index,
            y=pred_df[mask].covidOccupiedMVBeds,
            mode="lines+markers",
            marker=dict(size=8, opacity=0.7, symbol='circle'),
            line=dict(color=act_color, width=3),
            name="Actual COVID-19 MVBeds trends",
        )
    )

    # Predicted data plot
    line_styles = ["solid", "dash", "dot", "dashdot"]
    markers = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up']
    for col, display_col, line_style, marker in zip(forecast_columns, forecast_display_names, cycle(line_styles), cycle(markers)):
        fig.add_trace(
            go.Scatter(
                x=pred_df[mask].index,
                y=pred_df.loc[mask, col],
                mode="lines+markers",
                marker=dict(size=6, symbol=marker),
                line=dict(color=next(colors), width=2, dash=line_style),
                name=display_col,
            )
        )

    fig.update_layout(
        title=f"COVID-19 MVBeds Forecast Comparison for {selected_area}",
        title_font=dict(size=20),
        xaxis_title="Date",
        yaxis_title="COVID-19 MVBeds",
        xaxis=dict(title_font=dict(size=15), tickfont=dict(size=12)),
        yaxis=dict(title_font=dict(size=15), tickfont=dict(size=12)),
        legend=dict(
            font=dict(size=10),
            orientation="h",
            yanchor="top",
            y=1.02,  # Move the legend down to separate it from the title
            xanchor="center",
            x=0.5
        ),
        template="plotly_white",
        plot_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=10, r=40, t=80, b=40),  # Adjust left margin for better alignment
        width=800,
        height=400
    )

    if save_path:
        pio.write_image(fig, save_path)
    return fig


def highlight_abs_min(s, props=""):
    return np.where(s == np.nanmin(np.abs(s.values)), props, "")

In [3]:
# Load and Prepare Data
data_path = Path("data/hos_data/england_data.csv")
# data = pd.read_csv(data_path).drop("Unnamed: 0", axis=1)
data = pd.read_csv(data_path)
data["date"] = pd.to_datetime(data["date"])

# Select and Process Data
selected_area = "England"
# data_filtered = data[data["areaName"] == selected_area]
data_filtered = data.copy()
# Data Processing
data_filtered["date"] = pd.to_datetime(data_filtered["date"])
data_filtered.head()

,date,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,hospitalCases,newAdmissions,population,covidOccupiedMVBeds,cumAdmissions
0,2020-04-01,3989.0,694.0,35571.0,4730.0,15544.0,4382,56171302,0.0,31201
1,2020-04-02,3895.0,725.0,39466.0,5455.0,15364.0,3937,56171302,1878.0,35138
2,2020-04-03,3878.0,737.0,43344.0,6192.0,18158.0,3388,56171302,2261.0,38526
3,2020-04-04,3260.0,828.0,46604.0,7020.0,20489.0,3446,56171302,2458.0,41972
4,2020-04-05,2994.0,823.0,49598.0,7843.0,22051.0,3446,56171302,2650.0,45418


In [4]:
# 7 day rolling average for all columns except the target column (covidOccupiedMVBeds)
# Perform 7-day rolling average for all columns except 'covidOccupiedMVBeds'
target_column = 'covidOccupiedMVBeds'
rolling_window = 7

# Apply rolling mean to all columns except the target column
for column in data_filtered.columns:
    if column != target_column and column != 'date':
        data_filtered[column] = data_filtered[column].rolling(window=rolling_window, min_periods=1).mean()

data_filtered.head()

,date,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,hospitalCases,newAdmissions,population,covidOccupiedMVBeds,cumAdmissions
0,2020-04-01,3989.000000,694.000000,35571.000000,4730.00,15544.000000,4382.000000,56171302.0,0.0,31201.00
1,2020-04-02,3942.000000,709.500000,37518.500000,5092.50,15454.000000,4159.500000,56171302.0,1878.0,33169.50
2,2020-04-03,3920.666667,718.666667,39460.333333,5459.00,16355.333333,3902.333333,56171302.0,2261.0,34955.00
3,2020-04-04,3755.500000,746.000000,41246.250000,5849.25,17388.750000,3788.250000,56171302.0,2458.0,36709.25
4,2020-04-05,3603.200000,761.400000,42916.600000,6248.00,18321.200000,3719.800000,56171302.0,2650.0,38451.00


In [5]:
# # Load and Prepare Data
# data_path = Path("data/hos_data/merged_data.csv")
# data = pd.read_csv(data_path)

# data["date"] = pd.to_datetime(data["date"])

# # Select and Process Data
# selected_area = "London"
# data_filtered = data[data["nhs_region"] == selected_area]

# # Data Processing
# data_filtered["date"] = pd.to_datetime(data_filtered["date"])
# data_filtered.sort_values(by=["date", "areaName"], inplace=True)
# data_filtered.drop(
#     [
#         # "areaName",
#         # "areaCode",
#         "areaCode",
#         "region_x",
#         "areaType",
#         "nhs_region",
#         "cumAdmissions",
#         "cumulative_confirmed",
#         "cumulative_deceased",
#         "population",
#         "openstreetmap_id",
#         "latitude",
#         "longitude",
#         "region_y",
#         "epi_week",
#         # "latitude",
#         # "longitude",
#         # "epi_week",
#     ],
#     axis=1,
#     inplace=True,
# )
# data_filtered = data_filtered[::-1].reset_index(drop=True)


In [6]:
# Add rolling features
def add_rolling_features(df, window_size, columns, agg_funcs=None):
    if agg_funcs is None:
        agg_funcs = ["mean"]
    added_features = {}
    for column in columns:
        for func in agg_funcs:
            roll_col_name = f"{column}_rolling_{window_size}_{func}"
            df[roll_col_name] = df[column].rolling(window_size).agg(func)
            if column not in added_features:
                added_features[column] = []
            added_features[column].append(roll_col_name)
    df.dropna(inplace=True)
    return df, added_features


window_size = 7
columns_to_roll = ["hospitalCases", "newAdmissions", "new_confirmed", "new_deceased"]
agg_funcs = ["mean", "std"]

data_filtered, added_features = add_rolling_features(
    data_filtered, window_size, columns_to_roll, agg_funcs
)

for column, features in added_features.items():
    logging.info(f"{column}: {', '.join(features)}")


# Add time-lagged features
def add_lags(data, lags, features):
    added_features = []
    for feature in features:
        for lag in lags:
            new_feature = feature + f"_lag_{lag}"
            data[new_feature] = data[feature].shift(lag)
            added_features.append(new_feature)
    return data, added_features


lags = [1, 2, 3, 5, 7, 14, 21]
data_filtered, added_features = add_lags(data_filtered, lags, ["covidOccupiedMVBeds"])
data_filtered.dropna(inplace=True)


# Create temporal features
def create_temporal_features(df, date_column):
    df["month"] = df[date_column].dt.month
    df["day"] = df[date_column].dt.day
    df["day_of_week"] = df[date_column].dt.dayofweek
    return df


data_filtered = create_temporal_features(data_filtered, "date")
data_filtered = data_filtered.set_index("date")

2024-05-24 14:03:14,868 - INFO - hospitalCases: hospitalCases_rolling_7_mean, hospitalCases_rolling_7_std
2024-05-24 14:03:14,869 - INFO - newAdmissions: newAdmissions_rolling_7_mean, newAdmissions_rolling_7_std
2024-05-24 14:03:14,870 - INFO - new_confirmed: new_confirmed_rolling_7_mean, new_confirmed_rolling_7_std
2024-05-24 14:03:14,871 - INFO - new_deceased: new_deceased_rolling_7_mean, new_deceased_rolling_7_std


In [7]:
# # Load and process the SEIRD data
# seird_data = pd.read_csv(f"reports/output/pinn_{selected_area}_output.csv")
# seird_data["date"] = pd.to_datetime(seird_data["date"])
# seird_data.set_index("date", inplace=True)

# # Merge the two dataframes on the date index
# merged_data = pd.merge(data_filtered, seird_data, left_index=True, right_index=True, how="inner")

# # Drop rows with any missing values
# merged_data.dropna(inplace=True)

merged_data = data_filtered.copy()
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 868 entries, 2020-04-28 to 2022-09-12
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   new_confirmed                 868 non-null    float64
 1   new_deceased                  868 non-null    float64
 2   cumulative_confirmed          868 non-null    float64
 3   cumulative_deceased           868 non-null    float64
 4   hospitalCases                 868 non-null    float64
 5   newAdmissions                 868 non-null    float64
 6   population                    868 non-null    float64
 7   covidOccupiedMVBeds           868 non-null    float64
 8   cumAdmissions                 868 non-null    float64
 9   hospitalCases_rolling_7_mean  868 non-null    float64
 10  hospitalCases_rolling_7_std   868 non-null    float64
 11  newAdmissions_rolling_7_mean  868 non-null    float64
 12  newAdmissions_rolling_7_std   868 non-null   

In [8]:
merged_data.columns

Index(['new_confirmed', 'new_deceased', 'cumulative_confirmed',
       'cumulative_deceased', 'hospitalCases', 'newAdmissions', 'population',
       'covidOccupiedMVBeds', 'cumAdmissions', 'hospitalCases_rolling_7_mean',
       'hospitalCases_rolling_7_std', 'newAdmissions_rolling_7_mean',
       'newAdmissions_rolling_7_std', 'new_confirmed_rolling_7_mean',
       'new_confirmed_rolling_7_std', 'new_deceased_rolling_7_mean',
       'new_deceased_rolling_7_std', 'covidOccupiedMVBeds_lag_1',
       'covidOccupiedMVBeds_lag_2', 'covidOccupiedMVBeds_lag_3',
       'covidOccupiedMVBeds_lag_5', 'covidOccupiedMVBeds_lag_7',
       'covidOccupiedMVBeds_lag_14', 'covidOccupiedMVBeds_lag_21', 'month',
       'day', 'day_of_week'],
      dtype='object')

In [9]:

# Set the target variable and make it stationary
target = "covidOccupiedMVBeds"
seasonal_period = 7
auto_stationary = AutoStationaryTransformer(seasonal_period=seasonal_period)
data_stat = auto_stationary.fit_transform(merged_data[[target]], freq="D")
merged_data[target] = data_stat.values

# Ensure the index is a DateTimeIndex
merged_data.index = pd.to_datetime(merged_data.index)

# Filter data between the specified dates
start_date = "2020-04-27"
end_date = "2022-12-31"
merged_data = merged_data[start_date:end_date]

min_date = merged_data.index.min()
max_date = merged_data.index.max()

# Calculate the range of dates
date_range = max_date - min_date
logging.info(f"Data ranges from {min_date} to {max_date} ({date_range.days} days)")

# Calculate split points
# total_days = date_range.days
# train_end = min_date + pd.Timedelta(days=int(total_days * 0.70))
# val_end = train_end + pd.Timedelta(days=int(total_days * 0.20))

# # Split the data into training, validation, and testing sets
# train = merged_data[merged_data.index <= train_end]
# val = merged_data[(merged_data.index > train_end) & (merged_data.index <= val_end)]
# test = merged_data[merged_data.index > val_end]

# Calculate split points
train_end = min_date + pd.DateOffset(months=20)
val_end = train_end + pd.DateOffset(months=4)
test_end = val_end + pd.DateOffset(months=4)

train = merged_data[merged_data.index <= train_end]
val = merged_data[(merged_data.index > train_end) & (merged_data.index <= val_end)]
test = merged_data[merged_data.index > val_end]

# Calculate the percentage of dates in each dataset
total_sample = len(merged_data)
train_sample = len(train) / total_sample * 100
val_sample = len(val) / total_sample * 100
test_sample = len(test) / total_sample * 100

print(f"Train: {train_sample:.2f}%, Validation: {val_sample:.2f}%, Test: {test_sample:.2f}%")
print(f"Train: {len(train)} samples, Validation: {len(val)} samples, Test: {len(test)} samples")
print(f"Max date in train: {train.index.max()}, Min date in validation: {val.index.min()}, Max date in test: {test.index.max()}")

# # Log the number of samples in each set
# logging.info(f"Training set: {len(train)} samples")
# logging.info(f"Validation set: {len(val)} samples")
# logging.info(f"Testing set: {len(test)} samples")

2024-05-24 14:03:15,254 - INFO - Data ranges from 2020-04-28 00:00:00 to 2022-09-12 00:00:00 (867 days)


Train: 70.28%, Validation: 13.94%, Test: 15.78%
Train: 610 samples, Validation: 121 samples, Test: 137 samples
Max date in train: 2021-12-28 00:00:00, Min date in validation: 2021-12-29 00:00:00, Max date in test: 2022-09-12 00:00:00


In [10]:
# Concatenate the DataFrames
sample_df = pd.concat([train, val, test])

# Convert all the feature columns to float32
for col in sample_df.columns:
    sample_df[col] = sample_df[col].astype("float32")

columns_to_select = [
    "covidOccupiedMVBeds",
    # "hospitalCases_rolling_7_mean",
    # "hospitalCases_rolling_7_std",
    # "newAdmissions_rolling_7_mean",
    # "newAdmissions_rolling_7_std",
    # "new_confirmed_rolling_7_mean",
    # "new_confirmed_rolling_7_std",
    # "new_deceased_rolling_7_mean",
    # "new_deceased_rolling_7_std",
    "covidOccupiedMVBeds_lag_1",
    "covidOccupiedMVBeds_lag_2",
    "covidOccupiedMVBeds_lag_3",
    "covidOccupiedMVBeds_lag_5",
    "covidOccupiedMVBeds_lag_7",
    "covidOccupiedMVBeds_lag_14",
    "covidOccupiedMVBeds_lag_21",
    "month",
    "day",
    "day_of_week",
]

sample_df = sample_df[columns_to_select]
cols = list(sample_df.columns)
cols.remove("covidOccupiedMVBeds")
sample_df = sample_df[cols + ["covidOccupiedMVBeds"]]

# Prepare DataModule for PyTorch Lightning
datamodule = TimeSeriesDataModule(
    data=sample_df,
    n_val=val.shape[0],
    n_test=test.shape[0],
    window=7,  # 7 days window
    horizon=1,  # single step
    normalize="global",  # normalizing the data
    batch_size=32,
    num_workers=0,
)
datamodule.setup()

# Initialize pred_df
pred_df = test.copy()

In [11]:
# Function to train and evaluate models
def train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record, pred_df):
    trainer = pl.Trainer(
        min_epochs=5,
        max_epochs=200,
        accelerator="gpu",
        devices=1,
        callbacks=[EarlyStopping(monitor="valid_loss", patience=20)],
    )
    trainer.fit(model, datamodule)

    predictions = trainer.predict(model, datamodule.test_dataloader())
    predictions = torch.cat(predictions).squeeze().detach().numpy()
    predictions = predictions * datamodule.train.std + datamodule.train.mean
    actuals = test["covidOccupiedMVBeds"].values

    assert actuals.shape == predictions.shape, "Mismatch in shapes between actuals and predictions"

    metrics = {
        "Algorithm": algorithm_name,
        "MAE": mae(actuals, predictions),
        "MSE": mse(actuals, predictions),
        "MASE": mase(actuals, predictions, train["covidOccupiedMVBeds"].values),
        "Forecast Bias": forecast_bias(actuals, predictions),
    }

    value_formats = ["{}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.2f}"]
    metrics = {
        key: format_.format(value)
        for key, value, format_ in zip(metrics.keys(), metrics.values(), value_formats)
    }

    pred_df_ = pd.DataFrame({algorithm_name: predictions}, index=test.index)
    pred_df = pred_df.join(pred_df_)
    metric_record.append(metrics)
    logging.info(metrics)
    return metric_record, pred_df

# Train Vanilla RNN
rnn_config = SingleStepRNNConfig(
    rnn_type="RNN",
    input_size=len(columns_to_select),
    hidden_size=128,
    num_layers=10,
    bidirectional=False,
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
algorithm_name = "Vanilla RNN"
metric_record, pred_df = train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record=[], pred_df=pred_df)

# Train Vanilla LSTM
rnn_config = SingleStepRNNConfig(
    rnn_type="LSTM",
    input_size=len(columns_to_select),
    hidden_size=128,
    num_layers=10,
    bidirectional=False,
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
algorithm_name = "Vanilla LSTM"
metric_record, pred_df = train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record, pred_df)

# Train Vanilla GRU
rnn_config = SingleStepRNNConfig(
    rnn_type="GRU",
    input_size=len(columns_to_select),
    hidden_size=128,
    num_layers=10,
    bidirectional=False,
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
algorithm_name = "Vanilla GRU"
metric_record, pred_df = train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record, pred_df)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024-05-24 14:03:15.936025: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-24 14:03:15.982265: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-24 14:03:16.669977: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name | Type    | Params
-------------------------

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

2024-05-24 14:03:45,776 - INFO - {'Algorithm': 'Vanilla GRU', 'MAE': '12.5928', 'MSE': '171.0675', 'MASE': '31.1662', 'Forecast Bias': '-12.59'}


In [12]:
param_bounds = {
    "rnn_type": ["RNN", "GRU", "LSTM"],
    "hidden_size": (32, 128),
    "num_layers": (5, 30),
    "bidirectional": [True, False]
}

initial_params = ["RNN", 32, 5, True]
initial_temp = 10

def objective(params):
    rnn_type, hidden_size, num_layers, bidirectional = params
    rnn_config = SingleStepRNNConfig(
        rnn_type=rnn_type,
        input_size=len(columns_to_select),
        hidden_size=hidden_size,
        num_layers=num_layers,
        bidirectional=bidirectional,
        learning_rate=1e-3
    )
    model = SingleStepRNNModel(rnn_config)
    model.float()

    trainer = pl.Trainer(
        min_epochs=5,
        max_epochs=200,
        accelerator="gpu",
        devices=1,
        callbacks=[EarlyStopping(monitor="valid_loss", patience=20)],
    )
    trainer.fit(model, datamodule)
    
    shutil.rmtree("lightning_logs")

    predictions = trainer.predict(model, datamodule.test_dataloader())
    predictions = torch.cat(predictions).squeeze().detach().numpy()
    predictions = predictions * datamodule.train.std + datamodule.train.mean

    actuals = test["covidOccupiedMVBeds"].values

    assert actuals.shape == predictions.shape, "Mismatch in shapes between actuals and predictions"

    return np.mean(np.abs(actuals - predictions))

def neighbor(params):
    if len(params) == 4:
        rnn_type, hidden_size, num_layers, bidirectional = params
        hidden_size = np.random.randint(*param_bounds["hidden_size"])
        num_layers = np.random.randint(*param_bounds["num_layers"])
        rnn_type = np.random.choice(param_bounds["rnn_type"])
        bidirectional = bool(np.random.choice(param_bounds["bidirectional"]))
        return [rnn_type, hidden_size, num_layers, bidirectional]
    elif len(params) == 6:
        encoder_type, decoder_type, hidden_size, num_layers, bidirectional, decoder_use_all_hidden = params
        hidden_size = np.random.randint(*param_bounds["hidden_size"])
        num_layers = np.random.randint(*param_bounds["num_layers"])
        encoder_type = np.random.choice(param_bounds["encoder_type"])
        decoder_type = np.random.choice(param_bounds["decoder_type"])
        bidirectional = bool(np.random.choice(param_bounds["bidirectional"]))
        decoder_use_all_hidden = bool(np.random.choice(param_bounds["decoder_use_all_hidden"]))
        return [encoder_type, decoder_type, hidden_size, num_layers, bidirectional, decoder_use_all_hidden]

def simulated_annealing(objective, initial_params, initial_temp, neighbor, n_iter, cooling_rate=0.20, verbose=True):
    current_params = initial_params
    current_cost = objective(current_params)
    best_params = current_params
    best_cost = current_cost
    temp = initial_temp
    cost_history = []

    for i in range(n_iter):
        candidate_params = neighbor(current_params)
        candidate_cost = objective(candidate_params)

        acceptance_probability = np.exp(-abs(candidate_cost - current_cost) / temp)

        if candidate_cost < current_cost or np.random.uniform() < acceptance_probability:
            current_params = candidate_params
            current_cost = candidate_cost

            if current_cost < best_cost:
                best_params = current_params
                best_cost = current_cost

        temp *= cooling_rate
        cost_history.append(best_cost)

        logging.info(f"Iteration: {i+1}, Best Cost: {best_cost:.4f}, Current Cost: {current_cost:.4f}, Temperature: {temp:.4f}")
        
                # Break early if the minimum cost has been constant for over 10 iterations
        if i > 20 and np.all(np.isclose(cost_history[-20:], cost_history[-1])):
            break

    return best_cost, best_params, cost_history

n_iter = 100
cooling_rate = 0.95

best_cost, best_params, cost_history = simulated_annealing(
    objective, initial_params, initial_temp, neighbor, n_iter, cooling_rate
)

logging.info(f"Best Parameters: {best_params}, Best Cost: {best_cost}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1, len(cost_history) + 1), y=cost_history, mode="lines"))
fig.update_layout(
    title="Simulated Annealing Optimization for Hyperparameter Tuning",
    xaxis_title="Iteration",
    yaxis_title="Best Cost",
    template="plotly_white",
)

# save_path = f"reports/figures/{selected_area}_sa_optimization.pdf"
# pio.write_image(fig, save_path)
fig.show()

rnn_config = SingleStepRNNConfig(
    rnn_type=best_params[0],
    input_size=len(columns_to_select),
    hidden_size=best_params[1],
    num_layers=best_params[2],
    bidirectional=best_params[3],
    learning_rate=1e-3,
)

model = SingleStepRNNModel(rnn_config)
algorithm_name = f"Optimized {rnn_config.rnn_type} (SA)"
metric_record, pred_df = train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record, pred_df)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name | Type    | Params
---------------------------------
0 | rnn  | RNN     | 28.0 K
1 | fc   | Linear  | 65    
2 | loss | MSELoss | 0     
---------------------------------
28.0 K    Trainable params
0         Non-trainable params
28.0 K    Total params
0.112     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
HORIZON = 1
WINDOW = 7

encoder_config = RNNConfig(
    input_size=len(columns_to_select),
    hidden_size=32,
    num_layers=5,
    bidirectional=True
)

decoder_config = RNNConfig(
    input_size=len(columns_to_select),
    hidden_size=32,
    num_layers=5,
    bidirectional=True
)

rnn2fc_config = Seq2SeqConfig(
    encoder_type="LSTM",
    decoder_type="FC",
    encoder_params=encoder_config,
    decoder_params={"window_size": WINDOW, "horizon": HORIZON},
    decoder_use_all_hidden=True,
    learning_rate=1e-3,
)

model = Seq2SeqModel(rnn2fc_config)
algorithm_name = "Seq2Seq LSTM"
metric_record, pred_df = train_and_evaluate(model, rnn2fc_config, datamodule, test, train, algorithm_name, metric_record, pred_df)

param_bounds = {
    "encoder_type": ["RNN", "GRU", "LSTM"],
    "decoder_type": ["FC"],
    "hidden_size": (32, 128),
    "num_layers": (5, 30),
    "bidirectional": [True, False],
    "decoder_use_all_hidden": [True, False],
}

initial_params = ["RNN", "FC", 32, 5, True, True]

initial_temp = 10

def objective(params):
    encoder_type, decoder_type, hidden_size, num_layers, bidirectional, decoder_use_all_hidden = params

    encoder_config = RNNConfig(
        input_size=len(columns_to_select),
        hidden_size=hidden_size,
        num_layers=num_layers,
        bidirectional=bidirectional
    )

    decoder_config = RNNConfig(
        input_size=len(columns_to_select),
        hidden_size=hidden_size,
        num_layers=num_layers,
        bidirectional=bidirectional
    )

    rnn2fc_config = Seq2SeqConfig(
        encoder_type=encoder_type,
        decoder_type=decoder_type,
        encoder_params=encoder_config,
        decoder_params={"window_size": WINDOW, "horizon": HORIZON},
        decoder_use_all_hidden=decoder_use_all_hidden,
        learning_rate=1e-3,
    )

    model = Seq2SeqModel(rnn2fc_config)

    trainer = pl.Trainer(
        min_epochs=5,
        max_epochs=100,
        accelerator="gpu",
        devices=1,
        callbacks=[EarlyStopping(monitor="valid_loss", patience=10)],
    )

    trainer.fit(model, datamodule)

    predictions = trainer.predict(model, datamodule.test_dataloader())
    predictions = torch.cat(predictions).squeeze().detach().numpy()
    predictions = predictions * datamodule.train.std + datamodule.train.mean

    actuals = test["covidOccupiedMVBeds"].values

    assert actuals.shape == predictions.shape, "Mismatch in shapes between actuals and predictions"

    return np.mean(np.abs(actuals - predictions))

best_cost, best_params, cost_history = simulated_annealing(
    objective, initial_params, initial_temp, neighbor, n_iter, cooling_rate
)

logging.info(f"Best Parameters: {best_params}, Best Cost: {best_cost}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1, len(cost_history) + 1), y=cost_history, mode="lines"))
fig.update_layout(
    title="Simulated Annealing Optimization Seq2Seq for Hyperparameter Tuning",
    xaxis_title="Iteration",
    yaxis_title="Best Cost",
    template="plotly_white",
)

# save_path = f"reports/figures/{selected_area}_sa_seq2seq_optimization.pdf"
# pio.write_image(fig, save_path)
fig.show()

encoder_type, decoder_type, hidden_size, num_layers, bidirectional, decoder_use_all_hidden = best_params

encoder_config = RNNConfig(
    input_size=len(columns_to_select),
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=bidirectional
)

decoder_config = RNNConfig(
    input_size=len(columns_to_select),
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=bidirectional
)

rnn2fc_config = Seq2SeqConfig(
    encoder_type=encoder_type,
    decoder_type=decoder_type,
    encoder_params=encoder_config,
    decoder_params={"window_size": WINDOW, "horizon": HORIZON},
    decoder_use_all_hidden=decoder_use_all_hidden,
    learning_rate=1e-3,
)

model = Seq2SeqModel(rnn2fc_config)
algorithm_name = f"Optimized Seq2Seq {encoder_type} (SA)"
metric_record, pred_df = train_and_evaluate(model, rnn2fc_config, datamodule, test, train, algorithm_name, metric_record, pred_df)

shutil.rmtree("lightning_logs")

metric_df = pd.DataFrame(metric_record)
metric_df[["MAE", "MSE", "MASE", "Forecast Bias"]] = metric_df[
    ["MAE", "MSE", "MASE", "Forecast Bias"]
].astype("float32")

final_metric_file = f"reports/metrics/{selected_area}_final_metrics.csv"
metric_df.to_csv(final_metric_file, index=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type    | Params
------------------------------------
0 | encoder | LSTM    | 111 K 
1 | decoder | Linear  | 449   
2 | loss    | MSELoss | 0     
------------------------------------
112 K     Trainable params
0         Non-trainable params
112 K     Total params
0.449     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

2024-05-24 13:18:04,474 - INFO - {'Algorithm': 'Optimized Seq2Seq RNN (SA)', 'MAE': '12.8688', 'MSE': '183.5656', 'MASE': '106.3116', 'Forecast Bias': '12.69'}


In [ ]:
forecast_columns = [
    "Vanilla RNN", 
    "Vanilla LSTM", 
    "Vanilla GRU", 
    "Optimized RNN (SA)", 
    "Seq2Seq LSTM", 
    "Optimized Seq2Seq RNN (SA)"
]

forecast_display_names = [
    "Vanilla RNN", 
    "Vanilla LSTM", 
    "Vanilla GRU", 
    "Optimized RNN (SA)", 
    "Seq2Seq LSTM", 
    "Optimized Seq2Seq RNN (SA)"
]

comparison_fig = plot_forecast(pred_df, forecast_columns, selected_area, forecast_display_names, save_path=f"reports/figures/{selected_area}_forecast_comparison.pdf")
comparison_fig.show()

In [ ]:
pred_df.head(2)

,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,hospitalCases,newAdmissions,population,covidOccupiedMVBeds,cumAdmissions,hospitalCases_rolling_7_mean,...,covidOccupiedMVBeds_lag_21,month,day,day_of_week,Vanilla RNN,Vanilla LSTM,Vanilla GRU,Optimized RNN (SA),Seq2Seq LSTM,Optimized Seq2Seq RNN (SA)
date,,,,,,,,,,,,,,,,,,,,,
2022-08-16,4767.0,101.0,19610478.0,162597.0,10549.0,1121,56171302,5.327409,1141282,9629.428571,...,169.0,8,16,1,21.996094,22.381958,22.894104,22.722717,22.168701,24.118652
2022-08-15,5514.0,119.0,19605711.0,162496.0,10922.0,1189,56171302,5.532719,1140161,9882.571429,...,159.0,8,15,0,27.086731,22.331482,22.770996,26.809753,21.836182,48.159363


In [ ]:
pred_df.to_csv(f"reports/results/{selected_area}_predictions.csv")